<a href="https://colab.research.google.com/github/sinhaabhishekgh/NN_networks/blob/master/transfer_learning_with_text_classification_with_gnews_20dim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import division, absolute_import, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as datasets
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0-rc2


In [2]:
print("eager is " , tf.executing_eagerly())
print("hub v " , hub.__version__)
print("ava ", "GPU " if tf.config.experimental.list_physical_devices('GPU') else 'NOT')

eager is  True
hub v  0.6.0
ava  GPU 


In [0]:
train_validation_set = datasets.Split.TRAIN.subsplit([6,4])

(train_data, validation_data) , test_data = datasets.load(name="imdb_reviews", as_supervised=True, split=(train_validation_set, datasets.Split.TEST))

In [7]:
train_example_batch, train_label_batch = next(iter(train_data.batch(10)))
train_example_batch

<tf.Tensor: id=229, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [8]:
train_label_batch

<tf.Tensor: id=230, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [9]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], trainable=True, dtype=tf.string)
hub_layer(train_example_batch[:6])

<tf.Tensor: id=412, shape=(6, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ],
       [ 3.9022908 , -5.336768  ,  4.6433706 , -2.322609  , -5.6

In [12]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(train_data.shuffle(10000).batch(512), epochs=20, verbose=1, validation_data=validation_data.batch(512))

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


30/30 [==============================] - 9s 303ms/step - loss: 0.9571 - accuracy: 0.5245 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 7s 241ms/step - loss: 0.6933 - accuracy: 0.6031 - val_loss: 0.6311 - val_accuracy: 0.6478
Epoch 3/20
30/30 [==============================] - 7s 241ms/step - loss: 0.6087 - accuracy: 0.6799 - val_loss: 0.5819 - val_accuracy: 0.6964
Epoch 4/20
30/30 [==============================] - 7s 246ms/step - loss: 0.5615 - accuracy: 0.7227 - val_loss: 0.5459 - val_accuracy: 0.7309
Epoch 5/20
30/30 [==============================] - 7s 242ms/step - loss: 0.5214 - accuracy: 0.7525 - val_loss: 0.5119 - val_accuracy: 0.7556
Epoch 6/20
30/30 [==============================] - 7s 245ms/step - loss: 0.4877 - accuracy: 0.7797 - val_loss: 0.4862 - val_accuracy: 0.7742
Epoch 7/20
30/30 [==============================] - 7s 243ms/step - loss: 0.4571 - accuracy: 0.7991 - val_loss: 0.4631 - val_accuracy: 0.7863
Epoch 8/2

In [15]:
results = model.evaluate(test_data.batch(512), verbose=2)

49/49 - 5s - loss: 0.3188 - accuracy: 0.8619


[0.3187930152124288, 0.86188]

In [22]:
for name, value in zip(model.metrics_names, results):
  print("%s and %.3f" %  (name, value))

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))
  

loss and 0.319
accuracy and 0.862
loss: 0.319
accuracy: 0.862


In [23]:
test_sample_data, test_sameple_label = next(iter(test_data.batch(10)))
test_sample_data

<tf.Tensor: id=4891, shape=(10,), dtype=string, numpy=
array([b"I've watched the movie actually several times. And what i want to say about it is the only thing that made this movie high rank was the Burak Altay's incredible performance, absolutely nothing but that. Not even those silly model named Deniz Akkaya and some of these popular names at times in the movie... Burak is definitely very talented i've seen a few jobs he made and been through. Even though this is kind of horror movie, he's doing really good job in comedy movies and also in dramas too. I bet most of you all saw Asmali Konak the movie and TV series, those two would go for an example... All i'm gonna say is you better watch out for the new works coming out from Burak then you'll see.. Keep the good work bro, much love..",
       b'If you love Japanese monster movies, you\'ll love this action packed battle pitting an alien invader, intent upon conquering the Earth, and a "Giant Robot" with an armory of super weaponry. T

In [24]:
test_sameple_label

<tf.Tensor: id=4892, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [0]:
str = "I wish I could have voted this movie a ten, it's that funny. If they had intended for it to be that funny I would have given it a ten. I have to give it a 1, but it's the funniest darn 1 you'll ever want to watch. See the giant blur flash across the screen! Where did it come from? What is it? It flies, it terrifies, it's electrifying, it's on strings! This bird has real personality. I was about ten when I saw it for the first time, and when Big Bird appeared on Sesame Street, I was sure they were one and the same!","

In [29]:
res = model.predict(test_sample_data, verbose=2)

10/1 - 0s


In [31]:
res

array([[0.95514846],
       [0.2294823 ],
       [0.99987507],
       [0.99835396],
       [0.688197  ],
       [0.9968624 ],
       [0.40349418],
       [0.991414  ],
       [0.9880343 ],
       [0.55720204]], dtype=float32)